## Import Packages

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import collections
import datetime
import hashlib
import sys

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pystac
import rasterio
import shapely
import tqdm
from pystac_client import Client
from shapely import wkt

pd.set_option("display.max_colwidth", 250)
sys.path.insert(0, "../")

from src.azure_wrap.ml_client_utils import (
    initialize_blob_service_client,
    initialize_ml_client,
)

ml_client = initialize_ml_client()
abs_client = initialize_blob_service_client(ml_client)


def x_intersects_any_in_y(x: gpd.GeoSeries, y: gpd.GeoSeries):
    """Uses geopandas' spatial index to figure out, for each geometry in x,
    whether it intersects with *any* geometry in y.

    Same as `x.intersects(y.union_all())` but much faster.
    """
    if len(y) == 0:
        return np.zeros(len(x), dtype=bool)
    matches, distances = y.sindex.nearest(x, return_all=False, return_distance=True)
    return distances == 0

# Get MGRS catalogue overlapping O&G producing

## Get World Regions

In [ ]:
# # get country boundaries
# !curl -L -o data/ne_110m_admin_0_countries.zip https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip
# !unzip -u data/ne_110m_admin_0_countries.zip -d data/ne_110m_admin_0_countries
# !rm data/ne_110m_admin_0_countries.zip

In [ ]:
world = gpd.read_file("data/ne_110m_admin_0_countries/")
world.columns

In [ ]:
# now we're going to map countries in the world dataframeto the EI regions
# This won't be pretty.
# Note: moved Turkey from Europe to Middle East
EI_countries_in_regions = {  # copy-pasted from spreadsheet
    "USA": ["United States of America"],
    "North America wo US": ["Canada", "Mexico"],
    "Europe": [
        "Denmark",
        "Germany",
        "Italy",
        "Netherlands",
        "Norway",
        "Poland",
        "Romania",
        "Ukraine",
        "United Kingdom",
        "Georgia",
    ],
    "CIS": [
        "Armenia",
        "Azerbaijan",
        "Belarus",
        "Kazakhstan",
        "Kyrgyzstan",
        "Moldova",
        "Russia",
        "Tajikistan",
        "Turkmenistan",
        "Uzbekistan",
    ],
    "Middle East": [
        "Bahrain",
        "Iran",
        "Iraq",
        "Israel",
        "Kuwait",
        "Oman",
        "Qatar",
        "Saudi Arabia",
        "Syria",
        "United Arab Emirates",
        "Jordan",
        "Lebanon",
        "Turkey",
    ],
    "Asia": [
        "Bangladesh",
        "Brunei",
        "China",
        "India",
        "Indonesia",
        "Malaysia",
        "Myanmar",
        "Pakistan",
        "Thailand",
        "Vietnam",
    ],
}

world["EI_region"] = world["CONTINENT"].map(
    {
        "Africa": "Africa",
        "Asia": "Asia",
        "Europe": "Europe",
        "North America": "North America wo US",
        "South America": "South and Central America",
        "Oceania": "Australia",
        "Seven seas (open ocean)": "",
        "Antarctica": "",
    }
)

world.loc[world["SUBREGION"] == "Central America", "EI_region"] = "South and Central America"
world.loc[world["REGION_WB"] == "Middle East & North Africa", "EI_region"] = "Middle East"
world.loc[world["CONTINENT"] == "Africa", "EI_region"] = "Africa"
world.loc[world["SUBREGION"] == "Australia and New Zealand", "EI_region"] = "Australia"

EI_country_to_region_map = {
    country: region for (region, country_list) in EI_countries_in_regions.items() for country in country_list
}
# check all EI countries are present in the world dataframe
world_countries_set = set(world.SOVEREIGNT)
for country in EI_country_to_region_map:
    if country == "Bahrain":
        # Bahrain is missing but it's tiny,
        # so let's ignore that
        continue
    world_countries_set.remove(country)  # this raises a KeyError if not present
EI_mapped_countries = world.SOVEREIGNT.map(EI_country_to_region_map)
world["EI_region"] = world.EI_region.where(EI_mapped_countries.isnull(), EI_mapped_countries)

world.EI_region.value_counts()

In [ ]:
world.plot("EI_region")

In [ ]:
EI_geometries = world.dissolve("EI_region")

## MGRS tiles

In [ ]:
# !curl -L -o data/S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml https://hls.gsfc.nasa.gov/wp-content/uploads/2016/03/S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml

In [ ]:
# mgrs_gdf = gpd.read_file("data/S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml", columns=["Name"])
mgrs_gdf = gpd.read_file(
    "data/S2A_OPER_GIP_TILPAR_MPC__20151209T095117_V20150622T000000_21000101T000000_B00.kml", engine="pyogrio"
)

In [ ]:
# for some reason each MGRS is a GeometryCollection containing polygons and points
# let's get rid of the stupid point
mgrs_gdf = mgrs_gdf.dissolve("Name")  # performs a union on the GeometryCollection

In [ ]:
# the international date line is a pain in the ass and I don't want to deal with it
mgrs_gdf = mgrs_gdf[~(mgrs_gdf.geometry.bounds["minx"] == -180)]
# remove tiles outside of +-60deg latitude
mgrs_gdf = mgrs_gdf[(mgrs_gdf.geometry.bounds.maxy < 60) & (mgrs_gdf.geometry.bounds.miny > -60)]

In [ ]:
mgrs_gdf.shape

In [ ]:
# mgrs_gdf.explore()

In [ ]:
# EI_geometries.explore()

In [ ]:
mgrs_centroid = gpd.GeoDataFrame(geometry=mgrs_gdf.geometry.centroid)
joined = gpd.sjoin(mgrs_centroid, EI_geometries.reset_index(), how="left", predicate="within")
mgrs_gdf["EI_region"] = joined[["EI_region"]]

In [ ]:
# gpd.sjoin(aux_gdf, mgrs_gdf, how="left", predicate="intersects")["Name"].unique()

In [ ]:
# # remove tiles in our Aux/SBR set
# aux_latlongs = pd.read_csv("../src/data/ancillary/ground_truth_plumes.csv")

# from shapely.geometry import Point

# aux_latlongs["geometry"] = aux_latlongs.apply(lambda row: Point(row["lon"], row["lat"]), axis=1)
# aux_gdf = gpd.GeoDataFrame(aux_latlongs, geometry="geometry", crs="EPSG:4326")  # CRS for lat/lon
# aux_mgrs = gpd.sjoin(aux_gdf, mgrs_gdf, how="left", predicate="intersects")
# # aux_mgrs_list = aux_mgrs.index_right.unique().tolist()  # 8 MGRS
# aux_mgrs_list = aux_mgrs["Name"].unique().tolist()  # 8 MGRS

# mgrs_gdf = mgrs_gdf[~mgrs_gdf.index.isin(aux_mgrs_list)]

In [ ]:
# remove tiles with no region
mgrs_has_region = mgrs_gdf.EI_region.str.len() > 0
mgrs_gdf = mgrs_gdf[mgrs_has_region]
mgrs_gdf.shape  # number of MGRS granules

In [ ]:
mgrs_gdf.plot("EI_region")
plt.title("MGRS tiles coloured by EI region")

## Classifying MGRS scenes into O&G producing

- Here we just define O&G producing as REDACTED fields.
- See Appendix on how to load additional producing datasets

In [ ]:
producing_union = # Union of all producing areas, TODO: Insert as we can't share the one we were using

In [ ]:
producing_by_region = EI_geometries.intersection(producing_union)
producing_by_region.plot()

In [ ]:
mgrs_intersects_producing = x_intersects_any_in_y(mgrs_gdf.geometry, producing_by_region.geometry)

# add that as a column in data frame
mgrs_gdf["OG_producing"] = mgrs_intersects_producing
mgrs_gdf["OG_producing"].value_counts()

In [ ]:
mgrs_gdf_overlapping = mgrs_gdf[mgrs_gdf["OG_producing"] == True].copy()
mgrs_gdf_overlapping.shape

In [ ]:
mgrs_gdf_overlapping["overlapping_area"] = mgrs_gdf_overlapping.intersection(producing_union).area

In [ ]:
# mgrs_gdf_overlapping.drop("description", axis=1).explore()

# Get a catalogue of S2 IDs following a target region distribution

## Get target distribution per region

- After feedback from Rob & Jack, we've come up with this split for our training tiles: Use 34% US, 24% ME, 12% Africa, 10% CIS, 5% NA without USA, 5% SMA, 5% Asia, 2.5% Australia, 2.5% EU
    - **Note:** This is still somewhat arbitrary 

In [ ]:
total_tiles = 3095
num_tiles_by_region = {
    "USA": int(np.round(total_tiles * 0.34)),
    "Middle East": int(np.round(total_tiles * 0.24)),
    "Africa": int(np.round(total_tiles * 0.12)),
    "CIS": int(np.round(total_tiles * 0.10)),
    "North America wo US": int(np.round(total_tiles * 0.05)),
    "South and Central America": int(np.round(total_tiles * 0.05)),
    "Asia": int(np.round(total_tiles * 0.05)),
    "Australia": int(np.round(total_tiles * 0.025)),
    "Europe": int(np.round(total_tiles * 0.025)),
}
num_tiles_by_region

In [ ]:
# mgrs_gdf[mgrs_gdf["OG_producing"]==True].explore()

In [ ]:
print(mgrs_gdf[mgrs_gdf["OG_producing"] == True].shape)
mgrs_gdf[mgrs_gdf["OG_producing"] == True].EI_region.value_counts()

## Selecting producing MGRS
- For each region, if we need N tiles and have M available MGRS tiles:
    - If N < M: Sample N tiles randomly
    - If N > M: Take all M tiles, then sample (N-M) additional tiles from the same pool, repeating as long as needed
- The sampling is made reproducible by:
    - Using a deterministic seed generated from the region name + base_seed + step
    - Step increases for each repeated sampling within a region
    - Base seed increments for each new region

In [ ]:
# Generate a deterministic seed using hashlib: Thanks chatgpt
# For the same mgrs_gdf, num_tiles_by_region, and total_tiles, we will select the same
# MGRS for each region
def generate_seed(region: str, base_seed: int, step: int = 0) -> int:
    region_bytes = region.encode("utf-8")  # Convert region name to bytes
    hash_object = hashlib.md5(region_bytes)  # Create an MD5 hash object
    region_hash = int(hash_object.hexdigest(), 16)  # Convert hash to an integer
    return (region_hash + base_seed + step) % (2**32)  # Keep seed within 32-bit range


mgrs_selected = []
seed_count = 3334
for region, num in num_tiles_by_region.items():
    mgrs_prod = mgrs_gdf[(mgrs_gdf.EI_region == region) & (mgrs_gdf["OG_producing"] == True)].copy()
    print(f"{region:25}: We have {len(mgrs_prod):3} and want {num:3}")
    if num < len(mgrs_prod):
        print(f"  Sampling {num}")
        # Use a unique seed for the initial sampling
        seed = generate_seed(region, seed_count, step=0)
        mgrs_selected.append(mgrs_prod.sample(num, random_state=seed))
    else:
        print(f"  Taking all {len(mgrs_prod)} IDs once")
        mgrs_selected.append(mgrs_prod)
        num = num - len(mgrs_prod)
        if num < len(mgrs_prod):
            print(f"  Sampling additional {num}")
            # Use a different unique seed for additional sampling
            seed = generate_seed(region, seed_count, step=1)
            mgrs_selected.append(mgrs_prod.sample(num, random_state=seed))
        else:
            print(f"  Taking all {len(mgrs_prod)} IDs once")
            mgrs_selected.append(mgrs_prod)
            num = num - len(mgrs_prod)
            if num < len(mgrs_prod):
                print(f"  Sampling additional {num}")
                # Use yet another unique seed for further sampling
                seed = generate_seed(region, seed_count, step=2)
                mgrs_selected.append(mgrs_prod.sample(num, random_state=seed))
            else:
                print(f"  Taking all {len(mgrs_prod)} IDs once")
                mgrs_selected.append(mgrs_prod)
                num = num - len(mgrs_prod)
                if num < len(mgrs_prod):
                    print(f"  Sampling additional {num}")
                    # Use yet another unique seed for further sampling
                    seed = generate_seed(region, seed_count, step=3)
                    mgrs_selected.append(mgrs_prod.sample(num, random_state=seed))
                else:
                    print(f"  Taking all {len(mgrs_prod)} IDs once")
                    mgrs_selected.append(mgrs_prod)
                    num = num - len(mgrs_prod)
                    if num < len(mgrs_prod):
                        print(f"  Sampling additional {num}")
                        # Use yet another unique seed for further sampling
                        seed = generate_seed(region, seed_count, step=3)
                        mgrs_selected.append(mgrs_prod.sample(num, random_state=seed))

    seed_count += 1  # Increment base seed for next region
mgrs_selected = pd.concat(mgrs_selected)
mgrs_selected = mgrs_selected.reset_index()
assert len(mgrs_selected) == total_tiles

In [ ]:
mgrs_selected.explore()

In [ ]:
# plt.figure(figsize=(10, 20))
# ax = plt.gca()
# world.plot("EI_region", ax=ax, alpha=0.1)
# mgrs_selected.plot(ax=ax, alpha=0.5)
# plt.title("Selected tiles in O&G producing areas")

In [ ]:
# how many unique MGRS?
mgrs_selected["Name"].nunique()

In [ ]:
# max number of samples for a given MGRS
mgrs_selected["Name"].value_counts().max()

In [ ]:
mgrs_selected = mgrs_selected.drop(
    [
        "description",
        "timestamp",
        "begin",
        "end",
        "altitudeMode",
        "tessellate",
        "extrude",
        "visibility",
        "drawOrder",
        "icon",
        "snippet",
    ],
    axis=1,
)

In [ ]:
# Save master MGRS set. This will later be split into 80% train/10% val/10% test
master_filename = "2025_02_11_MGRS_within_OG_revamp_3095_master_set"
mgrs_selected.to_file(f"../src/data/tiles/s2/csv_files/{master_filename}.geojson", driver="GeoJSON")
mgrs_selected.to_csv(f"../src/data/tiles/s2/csv_files/{master_filename}.csv", index=False)

## Selecting dates at random using the STAC catalog
- From `Selecting producing MGRS` we have selected our MGRS tiles and determined the number of samples we want from each. This info is stored in `num_samples[mgrs]` (see sampling fn, `sample_dates_for_mgrs`)
- We enforce temporal diversity using different year ranges:
    - 1 sample: 2016-2024
    - 2 samples: 2016-2020, then 2021-2024
    - 3 samples: 2016-2019, then 2020-2022, then 2023-2024
    - ...
- **Core Sampling Logic:**
    - Each MGRS tile gets a deterministic seed: `get_seed_from_string(f"{mgrs}_{mgrs_counter[mgrs]}")`. This ensures we select the same dates each time
    - We set initial quality thresholds of 95% max nodata, 98% max clouds
    - If selected image has high nodata/clouds, we sample additional image with stricter thresholds (40% each)
    - We have an additional check that we don't select 2023 dates for MGRS in our detection threshold "test" set

In [ ]:
# Connect to AWS Sentinel-2 data
api_url = "https://planetarycomputer.microsoft.com/api/stac/v1"
import planetary_computer

client = Client.open(
    api_url,
    modifier=planetary_computer.sign_inplace,
)
collection = "sentinel-2-l2a"

# Load in current "test set" aka Detection Threshold tiles
det_thresh_mgrs = pd.read_csv("../src/data/tiles/s2/csv_files/fpr_areas.csv")
det_thresh_test_set_mgrs = det_thresh_mgrs.mgrs.unique().tolist()


### Sampling Logic Helper Functions ###
def get_seed_from_string(s: str) -> int:
    """Create reproducible seed from string using hashlib."""
    return int(hashlib.sha256(s.encode()).hexdigest(), 16) % (2**32)


def random_date(
    start: datetime.datetime, end: datetime.datetime, rng: np.random.Generator, mgrs: str
) -> datetime.datetime:
    while True:
        time_delta = int((end - start).total_seconds())
        random_seconds = int(rng.integers(0, time_delta))
        date = start + datetime.timedelta(seconds=random_seconds)

        # For test set MGRS, keep sampling until we get a non-2023 date
        if mgrs in det_thresh_test_set_mgrs and date.year == 2023:
            print("Got a test set MGRS, trying to find non 2023 date...")
            continue
        return date


def random_item_for_MGRS(
    mgrs: str, rng: np.random.Generator, max_cloud_cover: float = 60, start_year: int = 2016, end_year: int = 2024
):
    start_time = datetime.datetime(start_year, 1, 1)
    if start_year == 2016:
        # Start later to avoid not finding reference images
        end_range_start = datetime.datetime(start_year, 5, 1)
        end_range_end = datetime.datetime(end_year, 12, 31)
    else:
        end_range_start = datetime.datetime(start_year, 1, 1)
        end_range_end = datetime.datetime(end_year, 12, 31)

    end_time = random_date(end_range_start, end_range_end, rng, mgrs)

    search = client.search(
        collections=[collection],
        max_items=1,
        datetime=f"{start_time.isoformat()}/{end_time.isoformat()}",
        query={"s2:mgrs_tile": {"eq": mgrs}, "eo:cloud_cover": {"lt": max_cloud_cover}},
    )
    return next(search.items())


def item_area(item: pystac.Item) -> float:
    polygon = shapely.geometry.shape(item.geometry)
    return polygon.area

## within producing regions

In [ ]:
# Load in selected MGRS tiles (within producing regions)
mgrs_selected = gpd.read_file("../src/data/tiles/s2/csv_files/2025_02_11_MGRS_within_OG_revamp_3095_master_set.geojson")
mgrs_selected = mgrs_selected.sort_values("Name")
mgrs_we_want = mgrs_selected["Name"].value_counts().to_dict()
mgrs_selected.shape

In [ ]:
def sample_dates_for_mgrs(
    mgrs_selected_df: pd.DataFrame, num_samples: dict[str, int], debug: bool = True, verbose: bool = True
):
    # Output lists
    selected_prod_items = []
    nodata_percs = []
    additional_id_due_to_high_nodata_clouds = []

    def default_one():
        return 1

    mgrs_counter = collections.defaultdict(default_one)
    additional_id_counter = collections.defaultdict(int)
    already_sampled_dates = collections.defaultdict(set)
    already_sampled_cloud_covers = collections.defaultdict(list)
    already_sampled_nodatas = collections.defaultdict(list)

    for idx, row in tqdm.tqdm(mgrs_selected_df.iterrows(), total=len(mgrs_selected_df)):
        if debug:
            if idx == 30:
                break
        # Ensure that for any given MGRS tile, it will always select the same date,
        # so the output is reproducible
        mgrs = row["Name"]
        mgrs_area = row["geometry"].area

        # Create reproducible RNG for this MGRS tile and counter combination
        seed = get_seed_from_string(f"{mgrs}_{mgrs_counter[mgrs]}")
        rng = np.random.default_rng(seed)

        # Determine start and end year for temporal diversity
        if num_samples[mgrs] == 1:
            start_year = 2016
            end_year = 2024
        # If we want 2 IDs per tile, take one 2016-2021, one 2022-2024
        elif num_samples[mgrs] == 2:
            if mgrs_counter[mgrs] == 1:
                start_year = 2016
                end_year = 2020
            else:
                start_year = 2021
                end_year = 2024
        # If we want 3 IDs per tile, take one 2016-2019, one 2020-2022, one 2023-2024
        elif num_samples[mgrs] == 3:
            if mgrs_counter[mgrs] == 1:
                start_year = 2016
                end_year = 2019
            elif mgrs_counter[mgrs] == 2:
                start_year = 2020
                end_year = 2022
            else:
                start_year = 2023
                end_year = 2024
        # If we want 4 IDs per tile, take one 2016-2018, one 2019-2021, one 2022-2023, one 2024-2025
        elif num_samples[mgrs] == 4:
            if mgrs_counter[mgrs] == 1:
                start_year = 2016
                end_year = 2018
            elif mgrs_counter[mgrs] == 2:
                start_year = 2019
                end_year = 2021
            elif mgrs_counter[mgrs] == 3:
                start_year = 2022
                end_year = 2023
            else:
                start_year = 2024
                end_year = 2025
        # If we want 5 IDs per tile, take one 2016-2017, one 2018-2019, one 2020-2021, one 2022-2023, one 2024-2025
        elif num_samples[mgrs] == 5:
            if mgrs_counter[mgrs] == 1:
                start_year = 2016
                end_year = 2017
            elif mgrs_counter[mgrs] == 2:
                start_year = 2018
                end_year = 2019
            elif mgrs_counter[mgrs] == 3:
                start_year = 2020
                end_year = 2021
            elif mgrs_counter[mgrs] == 4:
                start_year = 2022
                end_year = 2023
            else:
                start_year = 2023
                end_year = 2024
        print(
            f"{mgrs}: We have {mgrs_counter[mgrs] - 1}/{num_samples[mgrs]} --> Sample randomly from {start_year}-01-01 to {end_year}-12-31"
        )

        # Initialize nodata and cloud cover thresholds
        max_nodata_px_perc = 95
        max_cloud_cover = 98
        for _ in range(100):  # 100 retries
            while True:
                try:
                    # Attempt to sample a random item
                    item = random_item_for_MGRS(
                        mgrs, rng=rng, max_cloud_cover=max_cloud_cover, start_year=start_year, end_year=end_year
                    )
                    id_ = item.to_dict()["properties"]["s2:product_uri"].replace(".SAFE", "")
                    if item.datetime in already_sampled_dates[mgrs]:
                        print(f"{mgrs} - {id_} --> DONT USE (we have this date already)")
                        continue
                    break
                except Exception:
                    if debug:
                        print("No data found between start and end date. Try again")
                    pass

            # Calculate nodata percentage and cloud cover
            nodata_perc = abs(100 - 100 * item_area(item) / mgrs_area)
            s2_date = id_.split("_")[2]
            s2_date = f"{s2_date[:4]}-{s2_date[4:6]}-{s2_date[6:8]}"
            cloud_perc = item.ext.eo.cloud_cover
            if verbose:
                print(
                    f"{mgrs} (#{mgrs_counter[mgrs]}) on {s2_date} has ({nodata_perc:6.1f}% nodata) and {cloud_perc:6.1f}% clouds ({id_})",
                    end="",
                )

            if nodata_perc > max_nodata_px_perc:
                print(" --> DONT USE (too much nodata)")
                continue
            if item.datetime in already_sampled_dates[mgrs]:
                print(" --> DONT USE (we have this date already)")
                continue
            else:
                print(" --> USE")
                selected_prod_items.append(item)
                nodata_percs.append(nodata_perc)
                additional_id_due_to_high_nodata_clouds.append(False)
                already_sampled_dates[mgrs].add(item.datetime)
                already_sampled_cloud_covers[mgrs].append(cloud_perc)
                already_sampled_nodatas[mgrs].append(nodata_perc)

                # sample another ID depending on how much nodata/clouds we got
                # Example: 90% nodata + 50% clouds ==> 95% of pixels are obscured ==> 95% to sample another ID with nodata max 40%/clouds max 40%
                random_ = rng.random() * 100
                if debug:
                    print(f"random: {random_:.2f} vs {nodata_perc + (1 - nodata_perc/100) * cloud_perc:.2f}")
                if random_ < nodata_perc + (1 - nodata_perc / 100) * cloud_perc:
                    max_nodata_px_perc = 40
                    max_cloud_cover = 40
                    while True:
                        try:
                            item = random_item_for_MGRS(
                                mgrs, rng=rng, max_cloud_cover=max_cloud_cover, start_year=start_year, end_year=end_year
                            )
                            id_ = item.to_dict()["properties"]["s2:product_uri"].replace(".SAFE", "")
                            if item.datetime in already_sampled_dates[mgrs]:
                                print(f"{mgrs} - {id_} --> DONT USE (we have this date already)")
                                continue
                            else:
                                nodata_perc = abs(100 - 100 * item_area(item) / mgrs_area)
                                s2_date = id_.split("_")[2]
                                s2_date = f"{s2_date[:4]}-{s2_date[4:6]}-{s2_date[6:8]}"
                                cloud_perc = item.ext.eo.cloud_cover
                                if verbose:
                                    print(
                                        f"  {mgrs} (#{mgrs_counter[mgrs]}) on {s2_date} has ({nodata_perc:6.1f}% nodata) and {cloud_perc:6.1f}% clouds ({id_})",
                                        end="",
                                    )
                                if nodata_perc > max_nodata_px_perc:
                                    print(" --> DONT USE (too much nodata)")
                                    continue
                            break
                        except Exception:
                            if debug:
                                print("No data found between start and end date. Try again")
                            pass
                    print(" --> USE")
                    additional_id_counter[mgrs] += 1
                    selected_prod_items.append(item)
                    nodata_percs.append(nodata_perc)
                    additional_id_due_to_high_nodata_clouds.append(True)
                    already_sampled_dates[mgrs].add(item.datetime)
                    already_sampled_cloud_covers[mgrs].append(cloud_perc)
                    already_sampled_nodatas[mgrs].append(nodata_perc)

                mgrs_counter[mgrs] += 1
                if num_samples[mgrs] == mgrs_counter[mgrs] - 1:
                    print(
                        f"{mgrs}: We sampled {num_samples[mgrs]} and {additional_id_counter[mgrs]} additional IDs due to "
                        f"high nodata/cloud cover ({sorted([k.date().isoformat() for k in already_sampled_dates[mgrs]])})"
                    )
                    print("#" * 150)
            break
        else:
            raise ValueError("couldn't find big enough tile after 100 tries")
    return selected_prod_items, nodata_percs, additional_id_due_to_high_nodata_clouds

In [ ]:
selected_prod_items, nodata_percs, additional_id_due_to_high_nodata_clouds = sample_dates_for_mgrs(
    mgrs_selected, mgrs_we_want, debug=False, verbose=True
)

In [ ]:
# Convert to df
selected_prod_df = pd.DataFrame(
    [
        {
            "mgrs": item.properties["s2:mgrs_tile"].zfill(
                5
            ),  # item.ext.grid.code.split("-")[1].zfill(5),  # 'MGRS-9UXA' -> '09UXA'
            "date": item.datetime.date().isoformat(),
            "datetime": item.datetime.isoformat(),
            "ID": item.to_dict()["properties"]["s2:product_uri"].replace(".SAFE", ""),
            "cloud_percent": item.ext.eo.cloud_cover,
        }
        for item in selected_prod_items
    ]
).sort_values(["mgrs", "date"])
selected_prod_df["selected_due_to_high_nodata_clouds"] = additional_id_due_to_high_nodata_clouds
selected_prod_df["nodata_perc"] = [abs(np.round(k, 3)) for k in nodata_percs]
print(selected_prod_df.shape)
selected_prod_df.sample(10)

In [ ]:
# Save final samples as csv
final_df_filename = f"2025_02_12_MGRS_with_IDs_within_OG_{len(selected_prod_df)}"
selected_prod_df.to_csv(
    f"../src/data/tiles/s2/csv_files/{final_df_filename}.csv",
    index=False,
)

### Final df Sanity Checks

In [ ]:
# Sanity Check: In our final MGRS-date pairs, MGRS that also belong to the det thresh csv
# should have non 2023 dates
selected_prod_df[selected_prod_df.mgrs.isin(det_thresh_test_set_mgrs)].date.apply(
    lambda d: pd.to_datetime(d).year
).unique()

In [ ]:
selected_prod_df["selected_due_to_high_nodata_clouds"].value_counts()

## Add more IDs for Hassi/Marcellus/Permian

In [ ]:
mgrs_selected = pd.read_csv("../src/data/tiles/s2/csv_files/2025_02_12_MGRS_with_IDs_within_OG_4547_with_metadata.csv")

if "EI_region" not in mgrs_selected.columns:
    # Join with mgrs_gdf to get region col
    mgrs_gdf_reset = mgrs_gdf.reset_index()
    mgrs_selected = pd.merge(
        mgrs_selected, mgrs_gdf[["EI_region", "geometry"]], left_on="mgrs", right_index=True, how="left"
    )

In [ ]:
region_geoms = [
    gpd.read_file("../src/data/ancillary/hassi.geojson").geometry.iloc[0].buffer(1.5).simplify(0.1),
    gpd.read_file("../src/data/ancillary/marcellus.geojson").geometry.iloc[0],
    gpd.read_file("../src/data/ancillary/permian.geojson").geometry.iloc[0],
]
regions = gpd.GeoDataFrame(
    {
        "region": [
            "hassi",
            "marcellus",
            "permian",
        ]
    },
    geometry=region_geoms,
    crs="EPSG:4326",
)
regions.explore()

In [ ]:
# Calculate overlap with hassi/marcellus/permian
mgrs_selected["geometry"] = mgrs_selected["geometry"].apply(wkt.loads)
mgrs_selected = gpd.GeoDataFrame(mgrs_selected, geometry="geometry", crs="EPSG:4326")
mgrs_selected["inside_val_regions"] = x_intersects_any_in_y(mgrs_selected.geometry, regions.geometry)
mgrs_selected["inside_hassi"] = x_intersects_any_in_y(
    mgrs_selected.geometry, regions[regions["region"] == "hassi"].geometry
)
mgrs_selected["inside_marcellus"] = x_intersects_any_in_y(
    mgrs_selected.geometry, regions[regions["region"] == "marcellus"].geometry
)
mgrs_selected["inside_permian"] = x_intersects_any_in_y(
    mgrs_selected.geometry, regions[regions["region"] == "permian"].geometry
)

mgrs_gdf["inside_val_regions"] = x_intersects_any_in_y(mgrs_gdf.geometry, regions.geometry)
mgrs_gdf["inside_hassi"] = x_intersects_any_in_y(mgrs_gdf.geometry, regions[regions["region"] == "hassi"].geometry)
mgrs_gdf["inside_marcellus"] = x_intersects_any_in_y(
    mgrs_gdf.geometry, regions[regions["region"] == "marcellus"].geometry
)
mgrs_gdf["inside_permian"] = x_intersects_any_in_y(mgrs_gdf.geometry, regions[regions["region"] == "permian"].geometry)

In [ ]:
mgrs_selected[["inside_hassi", "inside_marcellus", "inside_permian", "inside_val_regions"]].value_counts()

In [ ]:
val_ids = mgrs_selected[mgrs_selected["inside_val_regions"] == True].index.tolist()
len(val_ids), len(set(val_ids))

In [ ]:
mgrs_gdf[["inside_hassi", "inside_marcellus", "inside_permian", "inside_val_regions"]].value_counts()

In [ ]:
val_ids = mgrs_gdf[mgrs_gdf["inside_val_regions"] == True].index.tolist()
len(val_ids), len(set(val_ids))

In [ ]:
we_want_ids_per_mgrs = 9
already_sampled_dates = collections.defaultdict(list)
mgrs_additional = []
cols = [
    "Name",
    "geometry",
    "EI_region",
    "OG_producing",
    "inside_val_regions",
    "inside_hassi",
    "inside_marcellus",
    "inside_permian",
]
for mgrs in mgrs_gdf[mgrs_gdf["inside_hassi"] == True].index.tolist():
    already_sampled_dates[mgrs] = mgrs_selected[mgrs_selected["mgrs"] == mgrs]["datetime"].tolist()
    print(f"{mgrs}: {len(already_sampled_dates[mgrs])=}")
    how_many = we_want_ids_per_mgrs - len(already_sampled_dates[mgrs])
    if how_many > 0:
        for k in range(how_many):
            mgrs_additional.append(mgrs_gdf[mgrs_gdf.index == mgrs].reset_index()[cols])
for idx, row in (
    mgrs_selected[mgrs_selected["inside_marcellus"] == True]["mgrs"].value_counts().reset_index().iterrows()
):
    mgrs = row["mgrs"]
    already_sampled_dates[mgrs] = mgrs_selected[mgrs_selected["mgrs"] == mgrs]["datetime"].tolist()
    print(f"{mgrs}: {len(already_sampled_dates[mgrs])=}")
    how_many = we_want_ids_per_mgrs - len(already_sampled_dates[mgrs])
    if how_many > 0:
        for k in range(how_many):
            mgrs_additional.append(mgrs_gdf[mgrs_gdf.index == mgrs].reset_index()[cols])
for idx, row in mgrs_selected[mgrs_selected["inside_permian"] == True]["mgrs"].value_counts().reset_index().iterrows():
    mgrs = row["mgrs"]
    already_sampled_dates[mgrs] = mgrs_selected[mgrs_selected["mgrs"] == mgrs]["datetime"].tolist()
    print(f"{mgrs}: {len(already_sampled_dates[mgrs])=}")
    how_many = we_want_ids_per_mgrs - len(already_sampled_dates[mgrs])
    if how_many > 0:
        for k in range(how_many):
            mgrs_additional.append(mgrs_gdf[mgrs_gdf.index == mgrs].reset_index()[cols])
mgrs_additional = pd.concat(mgrs_additional, ignore_index=True)
mgrs_additional.shape

In [ ]:
mgrs_additional[["inside_hassi", "inside_marcellus", "inside_permian"]].value_counts()

In [ ]:
### Sampling Logic Helper Functions ###
def get_seed_from_string(s: str) -> int:
    """Create reproducible seed from string using hashlib."""
    return 1 + int(hashlib.sha256(s.encode()).hexdigest(), 16) % (2**32)


def sample_dates_for_mgrs(mgrs_selected_df: pd.DataFrame, debug: bool = True, verbose: bool = True):
    # Output lists
    selected_prod_items = []
    nodata_percs = []
    additional_id_due_to_high_nodata_clouds = []

    def default_one():
        return 1

    mgrs_counter = collections.defaultdict(default_one)
    additional_id_counter = collections.defaultdict(int)
    already_sampled_cloud_covers = collections.defaultdict(list)
    already_sampled_nodatas = collections.defaultdict(list)
    start_year = 2016
    end_year = 2025

    for idx, row in tqdm.tqdm(mgrs_selected_df.iterrows(), total=len(mgrs_selected_df)):
        # Ensure that for any given MGRS tile, it will always select the same date,
        # so the output is reproducible
        mgrs = row["Name"]
        mgrs_area = row["geometry"].area

        # Create reproducible RNG for this MGRS tile and counter combination
        seed = get_seed_from_string(f"{mgrs}_{mgrs_counter[mgrs]}")
        rng = np.random.default_rng(seed + idx)

        print(f"{mgrs}: Sample randomly from {start_year}-05-01 to {end_year}-12-31")

        # Initialize nodata and cloud cover thresholds
        max_nodata_px_perc = 95
        max_cloud_cover = 98
        for _ in range(100):  # 100 retries
            while True:
                try:
                    # Attempt to sample a random item
                    item = random_item_for_MGRS(
                        mgrs, rng=rng, max_cloud_cover=max_cloud_cover, start_year=start_year, end_year=end_year
                    )
                    id_ = item.to_dict()["properties"]["s2:product_uri"].replace(".SAFE", "")
                    if item.datetime in already_sampled_dates[mgrs]:
                        print(f"{mgrs} - {id_} --> DONT USE (we have this date already)")
                        print(already_sampled_dates[mgrs])
                        continue
                    break
                except Exception as err:
                    print(err)
                    import traceback

                    print(traceback.print_exception(None, err, err.__traceback__))

            # Calculate nodata percentage and cloud cover
            nodata_perc = abs(100 - 100 * item_area(item) / mgrs_area)
            s2_date = id_.split("_")[2]
            s2_date = f"{s2_date[:4]}-{s2_date[4:6]}-{s2_date[6:8]}"
            cloud_perc = item.ext.eo.cloud_cover
            if verbose:
                print(
                    f"{mgrs} on {s2_date} has ({nodata_perc:6.1f}% nodata) and {cloud_perc:6.1f}% clouds ({id_})",
                    end="",
                )
            if nodata_perc > max_nodata_px_perc:
                print(" --> DONT USE (too much nodata)")
                continue
            if item.datetime in already_sampled_dates[mgrs]:
                print(" --> DONT USE (we have this date already)")
                continue
            else:
                print(" --> USE")
                selected_prod_items.append(item)
                nodata_percs.append(nodata_perc)
                additional_id_due_to_high_nodata_clouds.append(False)
                already_sampled_dates[mgrs].append(item.datetime)
                already_sampled_cloud_covers[mgrs].append(cloud_perc)
                already_sampled_nodatas[mgrs].append(nodata_perc)

                # sample another ID depending on how much nodata/clouds we got
                # Example: 90% nodata + 50% clouds ==> 95% of pixels are obscured ==> 95% to sample another ID with nodata max 40%/clouds max 40%
                random_ = rng.random() * 100
                if debug:
                    print(f"random: {random_:.2f} vs {nodata_perc + (1 - nodata_perc/100) * cloud_perc:.2f}")
                if random_ < nodata_perc + (1 - nodata_perc / 100) * cloud_perc:
                    max_nodata_px_perc = 40
                    max_cloud_cover = 40
                    while True:
                        try:
                            item = random_item_for_MGRS(
                                mgrs, rng=rng, max_cloud_cover=max_cloud_cover, start_year=start_year, end_year=end_year
                            )
                            id_ = item.to_dict()["properties"]["s2:product_uri"].replace(".SAFE", "")
                            if item.datetime in already_sampled_dates[mgrs]:
                                print(f"{mgrs} - {id_} --> DONT USE (we have this date already)")
                                continue
                            else:
                                nodata_perc = abs(100 - 100 * item_area(item) / mgrs_area)
                                s2_date = id_.split("_")[2]
                                s2_date = f"{s2_date[:4]}-{s2_date[4:6]}-{s2_date[6:8]}"
                                cloud_perc = item.ext.eo.cloud_cover
                                if verbose:
                                    print(
                                        f"  {mgrs} on {s2_date} has ({nodata_perc:6.1f}% nodata) and {cloud_perc:6.1f}% clouds ({id_})",
                                        end="",
                                    )
                                if nodata_perc > max_nodata_px_perc:
                                    print(" --> DONT USE (too much nodata)")
                                    continue
                            break
                        except Exception as err:
                            print(err)
                            import traceback

                            print(traceback.print_exception(None, err, err.__traceback__))
                    print(" --> USE")
                    additional_id_counter[mgrs] += 1
                    selected_prod_items.append(item)
                    nodata_percs.append(nodata_perc)
                    additional_id_due_to_high_nodata_clouds.append(True)
                    already_sampled_dates[mgrs].append(item.datetime)
                    already_sampled_cloud_covers[mgrs].append(cloud_perc)
                    already_sampled_nodatas[mgrs].append(nodata_perc)
            break
        else:
            raise ValueError("couldn't find big enough tile after 100 tries")
    return selected_prod_items, nodata_percs, additional_id_due_to_high_nodata_clouds


selected_mgrs_additional, nodata_percs, additional_id_due_to_high_nodata_clouds = sample_dates_for_mgrs(
    mgrs_additional, debug=True, verbose=True
)

In [ ]:
# Convert to df
additional_df = pd.DataFrame(
    [
        {
            "mgrs": item.properties["s2:mgrs_tile"].zfill(5),
            "date": item.datetime.date().isoformat(),
            "datetime": item.datetime.isoformat(),
            "ID": item.to_dict()["properties"]["s2:product_uri"].replace(".SAFE", ""),
            "cloud_percent": item.ext.eo.cloud_cover,
        }
        for item in selected_mgrs_additional
    ]
).sort_values(["mgrs", "date"])
additional_df["selected_due_to_high_nodata_clouds"] = additional_id_due_to_high_nodata_clouds
additional_df["nodata_perc"] = [abs(np.round(k, 3)) for k in nodata_percs]
print(additional_df.shape)
additional_df.sample(10)

In [ ]:
# IDs we have already sampled
mgrs_selected.loc[
    mgrs_selected["inside_val_regions"] == True,
    ["mgrs", "date", "datetime", "ID", "cloud_percent", "selected_due_to_high_nodata_clouds", "nodata_perc"],
].shape

In [ ]:
# Add new IDs to get the dataframe of hassi/marcellus/permian IDs we will use for train/val
val_mgrs = pd.concat(
    (
        mgrs_selected.loc[
            mgrs_selected["inside_val_regions"] == True,
            ["mgrs", "date", "datetime", "ID", "cloud_percent", "selected_due_to_high_nodata_clouds", "nodata_perc"],
        ].copy(),
        additional_df,
    ),
    ignore_index=True,
)

val_mgrs.shape

In [ ]:
if "EI_region" not in val_mgrs.columns:
    # Join with mgrs_gdf to get region col
    val_mgrs = pd.merge(val_mgrs, mgrs_gdf[["EI_region", "geometry"]], left_on="mgrs", right_index=True, how="left")

val_mgrs = gpd.GeoDataFrame(val_mgrs, geometry="geometry", crs="EPSG:4326")
val_mgrs["inside_val_regions"] = x_intersects_any_in_y(val_mgrs.geometry, regions.geometry)
val_mgrs["inside_hassi"] = x_intersects_any_in_y(val_mgrs.geometry, regions[regions["region"] == "hassi"].geometry)
val_mgrs["inside_marcellus"] = x_intersects_any_in_y(
    val_mgrs.geometry, regions[regions["region"] == "marcellus"].geometry
)
val_mgrs["inside_permian"] = x_intersects_any_in_y(val_mgrs.geometry, regions[regions["region"] == "permian"].geometry)

In [ ]:
val_mgrs["inside_val_regions"].value_counts()

In [ ]:
print(val_mgrs.loc[val_mgrs["inside_hassi"] == True, "mgrs"].nunique())
val_mgrs.loc[val_mgrs["inside_hassi"] == True, "mgrs"].value_counts()

In [ ]:
print(val_mgrs.loc[val_mgrs["inside_marcellus"] == True, "mgrs"].nunique())
val_mgrs.loc[val_mgrs["inside_marcellus"] == True, "mgrs"].value_counts()

In [ ]:
print(val_mgrs.loc[val_mgrs["inside_permian"] == True, "mgrs"].nunique())
val_mgrs.loc[val_mgrs["inside_permian"] == True, "mgrs"].value_counts()

In [ ]:
val_mgrs[["inside_hassi", "inside_marcellus", "inside_permian"]].value_counts()

In [ ]:
# Save final samples as csv
val_mgrs.to_csv(
    "../src/data/tiles/s2/csv_files/2025_02_19_MGRS_with_IDs_hassi_marc_permian_699.csv",
    index=False,
)

# Split into Val/Train
- Validation = 50% of Hassi/Marcellus/Permian
- Train = other 50% of H/M/P + all other global IDs

In [ ]:
val_mgrs = pd.read_csv("../src/data/tiles/s2/csv_files/2025_02_19_MGRS_with_IDs_hassi_marc_permian_699.csv")
print(val_mgrs.shape)
train_mgrs = pd.read_csv("../src/data/tiles/s2/csv_files/2025_02_12_MGRS_with_IDs_within_OG_4547_with_metadata.csv")
print(train_mgrs.shape)
train_mgrs = train_mgrs[~train_mgrs["mgrs"].isin(val_mgrs["mgrs"].tolist())]
print(train_mgrs.shape)

In [ ]:
# STRATIFY NODATA/CLOUDS/% PER REGION BETWEEN TRAIN AND VAL
best_error = 10000
best_seed = 0
for seed in range(0, 10000):
    val_region_mgrs = sorted(val_mgrs["mgrs"].unique())
    np.random.seed(seed)
    np.random.shuffle(val_region_mgrs)
    df_train = val_mgrs[val_mgrs["mgrs"].isin(val_region_mgrs[: int(0.5 * len(val_region_mgrs))])][
        [
            "mgrs",
            "date",
            "cloud_percent",
            "nodata_perc",
            "EI_region",
            "inside_hassi",
            "inside_marcellus",
            "inside_permian",
        ]
    ].copy()
    df_val = val_mgrs[val_mgrs["mgrs"].isin(val_region_mgrs[int(0.5 * len(val_region_mgrs)) :])][
        [
            "mgrs",
            "date",
            "cloud_percent",
            "nodata_perc",
            "EI_region",
            "inside_hassi",
            "inside_marcellus",
            "inside_permian",
        ]
    ].copy()
    hassi_error = np.std([df_val["inside_hassi"].mean(), df_train["inside_hassi"].mean()]) / np.mean(
        [df_val["inside_hassi"].mean(), df_train["inside_hassi"].mean()]
    )
    marcellus_error = np.std([df_val["inside_marcellus"].mean(), df_train["inside_marcellus"].mean()]) / np.mean(
        [df_val["inside_marcellus"].mean(), df_train["inside_marcellus"].mean()]
    )
    permian_error = np.std([df_val["inside_permian"].mean(), df_train["inside_permian"].mean()]) / np.mean(
        [df_val["inside_permian"].mean(), df_train["inside_permian"].mean()]
    )

    df_val_ = df_val[df_val["inside_hassi"] == True]
    mean_clouds_val = df_val_["cloud_percent"].mean()
    df_train_ = df_train[df_train["inside_hassi"] == True]
    mean_clouds_train = df_train_["cloud_percent"].mean()
    hassi_clouds_error = np.std([mean_clouds_val, mean_clouds_train]) / np.mean([mean_clouds_val, mean_clouds_train])

    df_val_ = df_val[df_val["inside_marcellus"] == True]
    mean_clouds_val = df_val_["cloud_percent"].mean()
    df_train_ = df_train[df_train["inside_marcellus"] == True]
    mean_clouds_train = df_train_["cloud_percent"].mean()
    marcellus_clouds_error = np.std([mean_clouds_val, mean_clouds_train]) / np.mean(
        [mean_clouds_val, mean_clouds_train]
    )

    df_val_ = df_val[df_val["inside_permian"] == True]
    mean_clouds_val = df_val_["cloud_percent"].mean()
    df_train_ = df_train[df_train["inside_permian"] == True]
    mean_clouds_train = df_train_["cloud_percent"].mean()
    permian_clouds_error = np.std([mean_clouds_val, mean_clouds_train]) / np.mean([mean_clouds_val, mean_clouds_train])

    # NODATA
    df_val_ = df_val[df_val["inside_hassi"] == True]
    mean_clouds_val = df_val_["nodata_perc"].mean()
    df_train_ = df_train[df_train["inside_hassi"] == True]
    mean_clouds_train = df_train_["nodata_perc"].mean()
    # print(f'Nodata H: {mean_clouds_val:.3f} vs {mean_clouds_train:.3f}')
    hassi_nodata_error = np.std([mean_clouds_val, mean_clouds_train]) / np.mean([mean_clouds_val, mean_clouds_train])

    df_val_ = df_val[df_val["inside_marcellus"] == True]
    mean_clouds_val = df_val_["nodata_perc"].mean()
    df_train_ = df_train[df_train["inside_marcellus"] == True]
    mean_clouds_train = df_train_["nodata_perc"].mean()
    # print(f'Nodata M: {mean_clouds_val:.3f} vs {mean_clouds_train:.3f}')
    marcellus_nodata_error = np.std([mean_clouds_val, mean_clouds_train]) / np.mean(
        [mean_clouds_val, mean_clouds_train]
    )

    df_val_ = df_val[df_val["inside_permian"] == True]
    mean_clouds_val = df_val_["nodata_perc"].mean()
    df_train_ = df_train[df_train["inside_permian"] == True]
    mean_clouds_train = df_train_["nodata_perc"].mean()
    # print(f'Nodata P: {mean_clouds_val:.3f} vs {mean_clouds_train:.3f}')
    permian_nodata_error = np.std([mean_clouds_val, mean_clouds_train]) / np.mean([mean_clouds_val, mean_clouds_train])

    perc_mult = 4
    total_error = (
        perc_mult * hassi_error
        + perc_mult * marcellus_error
        + perc_mult * permian_error
        + hassi_clouds_error
        + marcellus_clouds_error
        + permian_clouds_error
        + hassi_nodata_error
        + marcellus_nodata_error
        + permian_nodata_error
    )
    if total_error < best_error:
        best_error = total_error
        best_seed = seed
        print(
            f"{seed:5}: {total_error:.2f} = Hassi% {perc_mult*hassi_error:.2f} + Marcellus% {perc_mult*marcellus_error:.2f} + Permian% {perc_mult*permian_error:.2f} "
            f"-- CLOUDS H {hassi_clouds_error:.2f} + M {marcellus_clouds_error:.2f} + P {permian_clouds_error:.2f}"
            f"-- NODATA H {hassi_nodata_error:.2f} + M {marcellus_nodata_error:.2f} + P {permian_nodata_error:.2f}"
        )
    # break

In [ ]:
# PRINT STATS FOR BEST SPLIT
best_seed = 6002
for seed in [best_seed]:
    print(f"Seed {seed}")
    val_region_mgrs = sorted(val_mgrs["mgrs"].unique())
    np.random.seed(seed)
    np.random.shuffle(val_region_mgrs)
    df_train = val_mgrs[val_mgrs["mgrs"].isin(val_region_mgrs[: int(0.5 * len(val_region_mgrs))])].copy()
    df_val = val_mgrs[val_mgrs["mgrs"].isin(val_region_mgrs[int(0.5 * len(val_region_mgrs)) :])].copy()
    print(
        f'Val   H: {df_val["inside_hassi"].mean():.3f}, M: {df_val["inside_marcellus"].mean():.3f}, P: {df_val["inside_permian"].mean():.3f}'
    )
    print(
        f'Train H: {df_train["inside_hassi"].mean():.3f}, M: {df_train["inside_marcellus"].mean():.3f}, P: {df_train["inside_permian"].mean():.3f}'
    )

    df_val_ = df_val[df_val["inside_hassi"] == True]
    mean_clouds_val = df_val_["cloud_percent"].mean()
    df_train_ = df_train[df_train["inside_hassi"] == True]
    mean_clouds_train = df_train_["cloud_percent"].mean()
    print(f"Clouds H: {mean_clouds_val:.3f} vs {mean_clouds_train:.3f}")

    df_val_ = df_val[df_val["inside_marcellus"] == True]
    mean_clouds_val = df_val_["cloud_percent"].mean()
    df_train_ = df_train[df_train["inside_marcellus"] == True]
    mean_clouds_train = df_train_["cloud_percent"].mean()
    print(f"Clouds M: {mean_clouds_val:.3f} vs {mean_clouds_train:.3f}")

    df_val_ = df_val[df_val["inside_permian"] == True]
    mean_clouds_val = df_val_["cloud_percent"].mean()
    df_train_ = df_train[df_train["inside_permian"] == True]
    mean_clouds_train = df_train_["cloud_percent"].mean()
    print(f"Clouds P: {mean_clouds_val:.3f} vs {mean_clouds_train:.3f}")

    # NODATA
    df_val_ = df_val[df_val["inside_hassi"] == True]
    mean_clouds_val = df_val_["nodata_perc"].mean()
    df_train_ = df_train[df_train["inside_hassi"] == True]
    mean_clouds_train = df_train_["nodata_perc"].mean()
    print(f"Nodata H: {mean_clouds_val:.3f} vs {mean_clouds_train:.3f}")

    df_val_ = df_val[df_val["inside_marcellus"] == True]
    mean_clouds_val = df_val_["nodata_perc"].mean()
    df_train_ = df_train[df_train["inside_marcellus"] == True]
    mean_clouds_train = df_train_["nodata_perc"].mean()
    print(f"Nodata M: {mean_clouds_val:.3f} vs {mean_clouds_train:.3f}")

    df_val_ = df_val[df_val["inside_permian"] == True]
    mean_clouds_val = df_val_["nodata_perc"].mean()
    df_train_ = df_train[df_train["inside_permian"] == True]
    mean_clouds_train = df_train_["nodata_perc"].mean()
    print(f"Nodata P: {mean_clouds_val:.3f} vs {mean_clouds_train:.3f}")

In [ ]:
df_train.shape, df_val.shape

In [ ]:
# Prepare global IDs and val region IDs selected for train to merge together
for col in ["OG_producing", "overlapping_area", "omni_nodata_perc", "omni_cloud_perc", "omni_cloud_shadow_perc"]:
    if col in train_mgrs.columns:
        train_mgrs = train_mgrs.drop(col, axis=1)
for col in ["inside_val_regions", "inside_hassi", "inside_marcellus", "inside_permian"]:
    train_mgrs[col] = False
print(train_mgrs.shape)

In [ ]:
df_train_final = pd.concat((train_mgrs, df_train), axis=0, ignore_index=True)
df_train_final.shape

In [ ]:
df_train_final["mgrs"].nunique()

In [ ]:
df_val.shape, df_val["mgrs"].nunique()

In [ ]:
# Save final samples as csv
df_train_final.to_csv(
    f"../src/data/tiles/s2/csv_files/2025_02_19_MGRS_with_IDs_within_OG_train_{len(df_train_final)}.csv",
    index=False,
)
df_val.to_csv(
    f"../src/data/tiles/s2/csv_files/2025_02_19_MGRS_with_IDs_within_OG_val_{len(df_val)}.csv",
    index=False,
)

In [ ]:
train_gdf = df_train_final.copy(deep=True)
train_gdf["geometry"] = train_gdf["geometry"].apply(wkt.loads)
train_gdf = gpd.GeoDataFrame(train_gdf, geometry="geometry", crs="EPSG:4326")

val_gdf = df_val.copy(deep=True)
val_gdf["geometry"] = val_gdf["geometry"].apply(wkt.loads)
val_gdf = gpd.GeoDataFrame(val_gdf, geometry="geometry", crs="EPSG:4326")
print(val_gdf.shape)

m = train_gdf.explore(color="green", name="Train", legend=True)
m = val_gdf.explore(m=m, color="red", name="Val", legend=True)
m

# (OLD: Split into Train/Val Sets)
- Stratify by region%, seasonal distribution, OMNI cloud, OMNI cloud shadow, nodata, producing area overlap

#### Add OmniCloud cloud% and cloud shadow% metadata for each ID

In [ ]:
mgrs_selected

In [ ]:
%%time

### Add all relevant metadata to each row in csv ###
import tempfile
from pathlib import Path

from src.azure_wrap.ml_client_utils import download_blob_directly, initialize_blob_service_client, initialize_ml_client

ml_client = initialize_ml_client()
abs_client = initialize_blob_service_client(ml_client)

mgrs_selected = pd.read_csv("../src/data/tiles/s2/csv_files/2025_02_12_MGRS_with_IDs_within_OG_4547_with_metadata.csv")

if "EI_region" not in mgrs_selected.columns:
    # Join with mgrs_gdf to get region col
    mgrs_gdf_reset = mgrs_gdf.reset_index()
    mgrs_selected = pd.merge(
        mgrs_selected, mgrs_gdf[["EI_region", "geometry"]], left_on="mgrs", right_index=True, how="left"
    )

In [ ]:
# ! convert both gdf to area CRS for area to be meaningful
mgrs_equal_area = mgrs_gdf.to_crs("EPSG:6933")  # Mollweide projection
producing_equal_area = producing_by_region.to_crs("EPSG:6933")

producing_union_equal_area = producing_equal_area.unary_union
mgrs_equal_area["overlapping_area"] = mgrs_equal_area.geometry.intersection(producing_union_equal_area).area

# Transfer the calculated area back to original GDF
mgrs_gdf = mgrs_gdf.assign(overlapping_area=mgrs_equal_area["overlapping_area"])

# Merge overlap data into mgrs_selected
mgrs_selected = pd.merge(
    mgrs_selected, mgrs_gdf[["OG_producing", "overlapping_area"]], left_on="mgrs", right_index=True, how="left"
)


def return_omnicloud_for_s2_id(s2_id):
    s2_id_date = s2_id.split("_")[2].split("T")[0]
    year = str(int(s2_id_date[:4]))
    month = str(int(s2_id_date[4:6]))
    day = str(int(s2_id_date[6:8]))
    tile = s2_id.split("_")[-2][1:]
    tile_part1 = tile[:2]
    tile_part2 = tile[2:3]
    tile_part3 = tile[3:5]

    prefix_parts = ["tiles", tile_part1, tile_part2, tile_part3, year, month, day, "0"]
    l1c_prefix = "/".join(prefix_parts)

    try:
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_path = Path(temp_dir)
            download_blob_directly(
                blob_name=f"{l1c_prefix}/OmniCloud.tif",
                # blob_name=f"{l1c_prefix}/SCL.tif",
                local_download_filepath=temp_path / "tmp.tif",
                blob_service_client=abs_client,
                container_name="l1c-data",
            )
            # print(f"Downloading OmniCloud for {s2_id} from {l1c_prefix}/OmniCloud.tif")
            with rasterio.open(temp_path / "tmp.tif") as ds:
                probs = ds.read()
                # print(probs.shape)
                print(f"OmniCloud exists for {s2_id} at {l1c_prefix}/OmniCloud.tif")
                return probs
    except Exception as err:
        print(f"OmniCloud is missing for {s2_id} at {l1c_prefix}/OmniCloud.tif")
        print(err)
        # import traceback
        # print(traceback.print_exception(None, err, err.__traceback__))


def process_omnicloud(s2_id):
    """Process OmniCloud data for a single S2 ID and return metrics"""
    try:
        probs = return_omnicloud_for_s2_id(s2_id)
        if probs is None:
            return pd.Series({"omni_nodata_perc": np.nan, "omni_cloud_perc": np.nan, "omni_cloud_shadow_perc": np.nan})

        probs = probs.astype(np.float32)
        valid_px = float((probs[0] != 255).sum())
        nodata_px = float((probs[0] == 255).sum())
        all_px = probs.shape[1] * probs.shape[2]
        probs[probs == 255] = np.nan

        omnicloud_cloud_t = 35
        omnicloud_shadow_t = 30
        # Calculate cloud and shadow masks
        clouds_omni = (probs[1, :, :] + probs[2, :, :] > omnicloud_cloud_t).astype(np.uint8)
        shadows_omni = ((probs[3, :, :] > omnicloud_shadow_t) & (clouds_omni != 1)).astype(np.uint8)

        # Calculate percentages
        return pd.Series(
            {
                "omni_nodata_perc": 100 * nodata_px / all_px,
                "omni_cloud_perc": 100 * clouds_omni.sum() / valid_px,
                "omni_cloud_shadow_perc": 100 * shadows_omni.sum() / valid_px,
            }
        )
    except Exception as e:
        print(f"Error processing {s2_id}: {e!s}")
        return pd.Series({"omni_nodata_perc": np.nan, "omni_cloud_perc": np.nan, "omni_cloud_shadow_perc": np.nan})

In [ ]:
# Add OMNI metrics
omni_results = mgrs_selected["ID"].apply(process_omnicloud)

mgrs_selected["omni_nodata_perc"] = omni_results["omni_nodata_perc"]
mgrs_selected["omni_cloud_perc"] = omni_results["omni_cloud_perc"]
mgrs_selected["omni_cloud_shadow_perc"] = omni_results["omni_cloud_shadow_perc"]

In [ ]:
# Fill NaNs with regional means
for col in ["omni_nodata_perc", "omni_cloud_perc", "omni_cloud_shadow_perc"]:
    mgrs_selected[col] = mgrs_selected.groupby("EI_region")[col].transform(lambda x: x.fillna(x.mean()))

mgrs_selected.head(1)

In [ ]:
# Save with metadata so we don't have to recompute the OMNI metrics
mgrs_selected.to_csv(
    "../src/data/tiles/s2/csv_files/2025_02_12_MGRS_with_IDs_within_OG_4547_with_metadata.csv", index=False
)

#### Build stratification sampling logic

In [ ]:
mgrs_selected = pd.read_csv("../src/data/tiles/s2/csv_files/2025_02_12_MGRS_with_IDs_within_OG_4547_with_metadata.csv")

In [ ]:
%%time
# Group by EI_region and calculate baseline statistics
mgrs_selected["datetime"] = pd.to_datetime(mgrs_selected["datetime"], format="%Y-%m-%dT%H:%M:%S.%f%z")


def get_season(month):
    if month in [12, 1, 2]:
        return "winter"
    elif month in [3, 4, 5]:
        return "spring"
    elif month in [6, 7, 8]:
        return "summer"
    else:
        return "fall"


def calculate_year_season_distribution(dates):
    return dates.apply(lambda x: f"{x.year}-{get_season(x.month)}").value_counts(normalize=True).to_dict()


# NOTE: use mean because median only focuses on the midpoint of the data
# mean captures distributional info
grouped_stats = (
    mgrs_selected.groupby("EI_region")
    .agg(
        overlapping_area_mean=("overlapping_area", "mean"),
        temporal_distribution=(
            "datetime",
            lambda x: x.apply(lambda date: get_season(date.month)).value_counts(normalize=True).to_dict(),
        ),
        omni_cloud_perc_mean=("omni_cloud_perc", "mean"),
        omni_cloud_perc_std=("omni_cloud_perc", "std"),
        omni_cloud_shadow_perc_mean=("omni_cloud_shadow_perc", "mean"),
        omni_cloud_shadow_perc_std=("omni_cloud_shadow_perc", "std"),
        omni_nodata_perc_mean=("omni_nodata_perc", "mean"),
        omni_nodata_perc_std=("omni_nodata_perc", "std"),
    )
    .reset_index()
)
grouped_stats

In [ ]:
def calculate_distribution_metrics(df: pd.DataFrame) -> dict:
    """
    Calculate distribution metrics for a dataset
    Returns dict of metrics by region
    """
    metrics = {}
    # Calculate region proportions in split dataset
    region_props = df["EI_region"].value_counts(normalize=True).to_dict()

    # Calculate metrics for each region
    for region in df["EI_region"].unique():
        region_df = df[df["EI_region"] == region]

        # Create year-season column
        def get_season(month):
            if month in [12, 1, 2]:
                return "winter"
            elif month in [3, 4, 5]:
                return "spring"
            elif month in [6, 7, 8]:
                return "summer"
            else:
                return "fall"

        # Create season distribution, only using season instead of year-season
        temporal_dist = (
            region_df["datetime"].apply(lambda x: get_season(x.month)).value_counts(normalize=True).to_dict()
        )
        metrics[region] = {
            "proportion": region_props[region],
            "temporal_dist": temporal_dist,
            "mean_omni_cloud": region_df["omni_cloud_perc"].mean(),
            "mean_omni_cloud_shadow": region_df["omni_cloud_shadow_perc"].mean(),
            "mean_omni_nodata": region_df["omni_nodata_perc"].mean(),
            "mean_overlap_area": region_df["overlapping_area"].mean(),
        }
    return metrics


def score_split(
    master_df: pd.DataFrame,
    train_df: pd.DataFrame,
    val_df: pd.DataFrame,
    metric_weights: dict,
    verbose=False,
    potency=1.0,
) -> float:
    """
    Score how well the split maintains distributions.
    Lower score is better.
    """
    train_metrics = calculate_distribution_metrics(train_df)
    val_metrics = calculate_distribution_metrics(val_df)

    scores = {"proportion": 0, "temporal": 0, "cloud": 0, "shadow": 0, "nodata": 0, "overlap": 0}

    all_regions = set(train_metrics.keys()) | set(val_metrics.keys())

    # These are region-specific thresholds for each metric to help ensure similar
    # representation of each region across the train/val/test sets. The thresholds
    # are informed by the baseline stats in `grouped_stats` and are iteratively
    # adjusted to achieve the best split.

    # A stricter threshold (e.g., 0.10 for mean_cloud in the USA) forces the mean
    # cloud coverage to be nearly identical across train/val/test splits, ensuring
    # consistent representation for regions where variability is naturally low.
    # In contrast, a higher threshold (e.g., 0.20 for mean_cloud in default)
    # indicates that the random splitting already achieves relatively even metrics
    # for this region, so the threshold is more lenient
    metrics_map = {
        "mean_omni_cloud": (
            "cloud",
            {
                "USA": 0.10,
                "Australia": 0.12,
                "North America wo US": 0.10,
                "CIS": 0.12,
                "South and Central America": 0.10,
                "Europe": 0.15,
                "default": 0.25,
            },
        ),
        "mean_overlap_area": ("overlap", {"USA": 0.15, "Europe": 0.15, "default": 0.20}),
        "proportion": (
            "proportion",
            {"USA": 0.08, "Middle East": 0.10, "CIS": 0.08, "Australia": 0.05, "Europe": 0.05, "default": 0.15},
        ),
        "mean_omni_cloud_shadow": ("overlap", {"USA": 0.15, "Europe": 0.15, "default": 0.20}),
        "mean_omni_nodata": ("overlap", {"USA": 0.15, "Europe": 0.15, "North America wo US": 0.10, "default": 0.20}),
    }

    for region in all_regions:
        if region not in train_metrics or region not in val_metrics:
            scores["proportion"] += 1000
            continue

        # Score region proportions. Use exponential penalty for increase weighting
        props = [
            train_metrics[region]["proportion"],
            val_metrics[region]["proportion"],
        ]
        mean_prop = np.mean(props)
        variation = np.std(props) / (mean_prop + 1e-6)
        scores["proportion"] += variation**potency

        if verbose:
            means = [
                train_metrics[region]["mean_omni_cloud"],
                val_metrics[region]["mean_omni_cloud"],
            ]
            print(
                f"{region:20}: %proportion {[np.round(k, 3) for k in props]}, {np.std(props)=:.2f}, {mean_prop=:.2f}, {variation=:.2f} **2--> {variation ** potency:.2f}"
            )

        # Score temporal distribution
        region_seasons = set(train_metrics[region]["temporal_dist"].keys()) | set(
            val_metrics[region]["temporal_dist"].keys()
        )

        for period in region_seasons:
            month_props = [
                train_metrics[region]["temporal_dist"].get(period, 0),
                val_metrics[region]["temporal_dist"].get(period, 0),
            ]
            if min(month_props) == 0:  # Penalize missing seasons
                missing_penalty = (
                    10 * potency * (1 / len(region_seasons))
                )  # divide by num of avail seasons for that region
                scores["temporal"] += missing_penalty
                continue

            mean_month_props = np.mean(month_props)
            variation = np.std(month_props) / (mean_month_props + 1e-6)  # ** potency
            scores["temporal"] += variation

            if verbose:
                print(
                    f"{region:25}: Period {period:10}: Means {[np.round(k, 3) for k in month_props]}, {np.std(month_props)=:.2f}, {mean_month_props=:.2f},   --> {variation:.2f}"
                )

        # Score other metrics
        remaining_metrics_to_score = {
            "mean_omni_cloud": "cloud",
            "mean_omni_cloud_shadow": "shadow",
            "mean_omni_nodata": "nodata",
            "mean_overlap_area": "overlap",
        }

        for metric, score_key in remaining_metrics_to_score.items():
            values = [train_metrics[region][metric], val_metrics[region][metric]]
            mean_value = np.mean(values)
            variation = np.std(values) / (mean_value + 1e-6)
            scores[score_key] += variation

    # Apply weights
    if verbose:
        for k, v in scores.items():
            print(f"{k:15} with weight {metric_weights.get(k, 1.0):.2f} = {metric_weights.get(k, 1.0) * v:.2f}")

    total_score = sum(metric_weights.get(k, 1.0) * v for k, v in scores.items())
    return total_score, scores

In [ ]:
from tabulate import tabulate


def print_split_comparison(train_df: pd.DataFrame, val_df: pd.DataFrame):
    """Print comparison of distributions across splits in tabular format"""
    train_metrics = calculate_distribution_metrics(train_df)
    val_metrics = calculate_distribution_metrics(val_df)

    # Get all unique regions across all splits
    all_regions = set(train_metrics.keys()) | set(val_metrics.keys())

    # Basic metrics table
    print("\nDistribution Metrics by Region and Split:")
    print("-" * 80)

    basic_metrics = []
    for region in all_regions:
        for split_name, metrics in zip(["Train", "Val"], [train_metrics, val_metrics], strict=False):
            basic_metrics.append(
                [
                    region,
                    split_name,
                    metrics[region]["proportion"],
                    metrics[region]["mean_omni_cloud"],
                    metrics[region]["mean_omni_cloud_shadow"],
                    metrics[region]["mean_omni_nodata"],
                    metrics[region]["mean_overlap_area"],
                ]
            )
    print(
        tabulate(
            basic_metrics,
            headers=["Region", "Split", "Prop", "OMNICloud%", "OMNICloudShadow%", "OMNINoData%", "OverlapArea"],
            floatfmt=".3f",
            tablefmt="pipe",  # Use pipe format for better readability
        )
    )
    # Temporal distribution table
    print("\nSeasonal Distribution by Region:")
    print("-" * 80)
    temporal_metrics = []
    for region in all_regions:
        # Get all year-seasons for this region across splits
        region_year_seasons = set(train_metrics[region]["temporal_dist"].keys()) | set(
            val_metrics[region]["temporal_dist"].keys()
        )
        for split_name, metrics in zip(["Train", "Val"], [train_metrics, val_metrics], strict=False):
            top_seasons = dict(
                sorted(
                    [(ys, metrics[region]["temporal_dist"].get(ys, 0)) for ys in region_year_seasons],
                    key=lambda x: x[1],
                    reverse=True,
                )
            )
            # Format as "season: XX.X%"
            top_seasons_str = ", ".join(f"{ys}: {v:.1%}" for ys, v in top_seasons.items())
            temporal_metrics.append([region, split_name, top_seasons_str])
    print(tabulate(temporal_metrics, headers=["Region", "Split", "Top Seasons (with %)"], tablefmt="pipe"))

In [ ]:
# Perform random splits and evaluate fitness
rng = np.random.default_rng(42)
best_score = float("inf")
best_split = None
num_seeds = 4000
train_frac = 0.9
val_frac = 0.1

# Get unique MGRS tiles
unique_mgrs = mgrs_selected["mgrs"].unique()

metric_weights = {
    "proportion": 10.0,
    "overlap": 4.0,
    "cloud": 5.0,
    "shadow": 2.0,
    "temporal": 0.1,
    "nodata": 2.0,
}
potency = 1.75
verbose = False

for trial in range(num_seeds):
    # Randomly assign MGRS to splits
    rng.shuffle(unique_mgrs)
    n_train = int(len(unique_mgrs) * train_frac)
    n_val = int(len(unique_mgrs) * val_frac)

    # Split MGRS tiles
    train_mgrs = unique_mgrs[:n_train]
    val_mgrs = unique_mgrs[n_train:]

    # Create dataframe splits
    train = mgrs_selected[mgrs_selected["mgrs"].isin(train_mgrs)]
    val = mgrs_selected[mgrs_selected["mgrs"].isin(val_mgrs)]

    # Score the split
    score, component_scores = score_split(mgrs_selected, train, val, metric_weights, verbose=verbose, potency=potency)

    if score < best_score:
        best_score = score
        best_split = (train, val)
        print(f"New best score: {best_score:.4f} at trial {trial}")
        print("  Component scores:")
        for metric, value in component_scores.items():
            print(f"\t{metric:15s}: {value * metric_weights[metric]:.4f}")
        print_split_comparison(train, val)
        score_split(mgrs_selected, train, val, metric_weights, verbose=True, potency=potency)


# Output the best split
train, val = best_split
print()
print("Best split achieved:")
print(f"Train size: {len(train)}")
print(f"Validation size: {len(val)}")
print_split_comparison(train, val)

In [ ]:
len(train), train["mgrs"].nunique()

In [ ]:
len(val), val["mgrs"].nunique()

In [ ]:
train.to_csv("../src/data/tiles/s2/csv_files/2025_02_12_MGRS_with_IDs_within_OG_4547_train_4098.csv", index=False)
val.to_csv("../src/data/tiles/s2/csv_files/2025_02_12_MGRS_with_IDs_within_OG_4547_val_449.csv", index=False)

# (OLD: Appendix: Plots to Share)

In [ ]:
train = pd.read_csv("../src/data/tiles/s2/csv_files/2025_02_12_MGRS_with_IDs_within_OG_4547_train_4098.csv")
val = pd.read_csv("../src/data/tiles/s2/csv_files/2025_02_12_MGRS_with_IDs_within_OG_4547_val_449.csv")

In [ ]:
train_gdf = train.copy(deep=True)
train_gdf["geometry"] = train_gdf["geometry"].apply(wkt.loads)
train_gdf = gpd.GeoDataFrame(train_gdf, geometry="geometry", crs="EPSG:4326")
train_gdf["fold"] = "train"
train_gdf = train_gdf.drop(columns=["datetime"])

val_gdf = val.copy(deep=True)
val_gdf["geometry"] = val_gdf["geometry"].apply(wkt.loads)
val_gdf = gpd.GeoDataFrame(val_gdf, geometry="geometry", crs="EPSG:4326")
val_gdf["fold"] = "val"
val_gdf = val_gdf.drop(columns=["datetime"])

m = train_gdf.explore(color="green", name="Train", legend=True)
m = val_gdf.explore(m=m, color="red", name="Val", legend=True)
m

In [ ]:
train_gdf["fold"] = "train"
val_gdf["fold"] = "val"
gdf = pd.concat((train_gdf, val_gdf), ignore_index=True)

# Save the GeoDataFrame to a GeoJSON file
output_path = "../src/data/tiles/s2/csv_files/2025_02_12_MGRS_with_IDs_within_OG_4547_with_metadata.geojson"
gdf.to_file(output_path, driver="GeoJSON")

# Appendix: Load other producing datasets

- For querying our DB programtically it is easiest to use your AWS VM instead of an Azure VM since there is some connection magic needed (Diehl is the expert on this)
- See Slide 21-25 in [Tile Selection + Reference tiles + Chip masking](https://docs.google.com/presentation/d/1p4pgQJjGQB4yssF7Wxs0aLuxp6xofJeKFjqrp1iUXSs/edit#slide=id.g3254c12f455_0_0) for details on the HIFLD/OGIM datasets

In [ ]:
### Connect to DB ###

# db_port = 5432
# db_name = "orbio"
# db_user = "readonly_user"
# db_host = "orbio-dev-db-ro.ccfdaoc3gfph.eu-central-1.rds.amazonaws.com"
# from getpass import getpass
# db_password = getpass("Read-only password (see https://git.orbio.earth/orbio/wiki/-/wikis/Playbooks/SQL-DBs-Access-&-Troubleshooting-(see-here-for-read-only-DBs!) ):")

# conn = sqlalchemy.create_engine(
#     f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
# )

### Query HIFLD Data ###

# query = """
# SELECT geom
# FROM source
# WHERE origin_dataset = %s
# """
# url = 'https://hifld-geoplatform.opendata.arcgis.com/datasets/geoplatform::oil-and-natural-gas-wells/explore?location=35.790450%2C-102.912136%2C4.22'

# hifld = gpd.read_postgis(query, con=conn, params=(url,), geom_col='geom')

### Query OGIM USA Data ###

# query = """
# SELECT geom
# FROM source
# WHERE origin_dataset = 'ogim'
#   AND (TYPE <> 'OIL AND NATURAL GAS PIPELINES' AND TYPE <> 'NATURAL GAS FLARING DETECTIONS')
# """

# ogim_usa = gpd.read_postgis(query, con=conn, geom_col='geom')

# Use the less cloudy validation IDs from Hassi/Marcellus/Permian for Detection Threshold
- Later: Use new chip selection and use also tiles with more nodata/clouds

In [ ]:
MGRS_DATE_DF = pd.read_csv("../src/data/tiles/s2/csv_files/fpr_areas.csv")
val_df = pd.read_csv("../src/data/tiles/s2/csv_files/2025_02_19_MGRS_with_IDs_within_OG_val_348.csv")
print(val_df.shape)
val_df = val_df[(val_df["cloud_percent"] < 20) & (val_df["nodata_perc"] < 2)]
MGRS_DATE_DF.shape, val_df.shape

In [ ]:
val_df["region"] = "hassi"
val_df.loc[val_df["inside_marcellus"] == True, "region"] = "marcellus"
val_df.loc[val_df["inside_permian"] == True, "region"] = "permian"
val_df["region"].value_counts()

In [ ]:
val_df[["mgrs", "date", "region", "cloud_percent", "nodata_perc", "geometry"]].to_csv(
    "../src/data/tiles/s2/csv_files/2025_03_03_fpr_areas_122_from_h_m_p_valset.csv"
)

In [ ]:
val_df.to_csv("../src/data/tiles/s2/csv_files/2025_03_03_fpr_areas_122_from_h_m_p_valset.csv")

In [ ]:
val_df["date"].apply(lambda x: x[5:7]).value_counts()

In [ ]:
val_df["date"].apply(lambda x: x[:4]).value_counts()

In [ ]:
val_df.groupby(["inside_hassi", "inside_marcellus", "inside_permian"])["mgrs"].nunique()

In [ ]:
val_df.groupby(["inside_hassi", "inside_marcellus", "inside_permian"])["mgrs"].count()

In [ ]:
MGRS_DATE_DF["date"].str.contains("2023-").value_counts()

In [ ]:
MGRS_DATE_DF.groupby("region")["mgrs"].nunique()

In [ ]:
MGRS_DATE_DF["region"].value_counts()

In [ ]:
MGRS_DATE_DF

In [ ]:
esri_attribution = "esri"
m = gpd.read_file("../src/data/ancillary/marcellus_large.geojson").explore(
    tiles="Esri.WorldImagery", attr=esri_attribution, style_kwds={"color": "green", "fillOpacity": 0.5}
)
gpd.read_file("../src/data/ancillary/hassi.geojson").explore(m=m, style_kwds={"color": "orange", "fillOpacity": 0.5})
gpd.read_file("../src/data/ancillary/permian.geojson").explore(m=m, style_kwds={"color": "red", "fillOpacity": 0.5})

mgrs_gdf[mgrs_gdf.index.isin(MGRS_DATE_DF["mgrs"])].explore(m=m, style_kwds={"color": "blue", "fillOpacity": 0.5})
mgrs_gdf[mgrs_gdf.index.isin(val_df["mgrs"])].explore(m=m, style_kwds={"color": "cyan", "fillOpacity": 0.5})
mgrs_gdf[mgrs_gdf.index.isin(["40SBH"])].explore(m=m, style_kwds={"color": "magenta", "fillOpacity": 0.5})
m